<a href="https://colab.research.google.com/github/Fulankeee/Credit-Risk-Early-Warning-Project/blob/main/notebooks/01_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation and Loading

In [2]:
# Install project dependencies
!pip install -r https://raw.githubusercontent.com/Fulankeee/Credit-Risk-Early-Warning-Project/main/requirements.txt

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# For data stored on google drive
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/MyDrive/raw_data/"
# -----------------------------------------------------------------------------------------------------------------------------------
# For data stored on Onedrive

# -----------------------------------------------------------------------------------------------------------------------------------
# # Local saved dataset and local environment
# import os
# import pandas as pd
# DATA_PATH = os.getenv("DATA_PATH", "/Users/siyicheng/Desktop/raw_data")
# df = pd.read_csv(os.path.join(DATA_PATH, "application_train.csv"))
# -----------------------------------------------------------------------------------------------------------------------------------
# Upload directly to google colab
# from google.colab import files
# files.upload()

Mounted at /content/drive


In [4]:
app_train = pd.read_csv(DATA_PATH + "application_train.csv")
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Create DB + load CSVs
import os, sqlite3
import pandas as pd

DATA_DIR = "/content/drive/MyDrive/raw_data"
DB_PATH = "/content/home_credit.db"

conn = sqlite3.connect(DB_PATH)

files = [
    "application_train.csv",
    "application_test.csv",
    "bureau.csv",
    "bureau_balance.csv",
    "credit_card_balance.csv",
    "installments_payments.csv",
    "POS_CASH_balance.csv",
    "previous_application.csv"
]

for f in files:
    path = os.path.join(DATA_DIR, f)
    table = f.replace(".csv","")
    print("Loading:", table)
    df = pd.read_csv(path)
    df.to_sql(table, conn, if_exists="replace", index=False)

print("Done. Tables created.")

Loading: application_train
Loading: application_test
Loading: bureau
Loading: bureau_balance
Loading: credit_card_balance
Loading: installments_payments
Loading: POS_CASH_balance
Loading: previous_application
Done. Tables created.


In [6]:
# Sanity check
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", conn)

,name
0,POS_CASH_balance
1,application_test
2,application_train
3,bureau
4,bureau_balance
5,credit_card_balance
6,installments_payments
7,previous_application


# Understand the tables

In [14]:
# Find the primary key 'SK_ID_CURR'

pd.read_sql("""
SELECT
  COUNT(*) AS n_rows,
  COUNT(DISTINCT SK_ID_CURR) AS n_customers
FROM application_train;
""", conn)

,n_rows,n_customers
0,307511,307511


In [15]:

pd.read_sql("""
SELECT
  AVG(CASE WHEN TARGET=1 THEN 1.0 ELSE 0 END) AS target_rate,
  SUM(CASE WHEN TARGET=1 THEN 1 ELSE 0 END) AS n_bad,
  COUNT(*) AS n_total
FROM application_train;
""", conn)


,target_rate,n_bad,n_total
0,0.080729,24825,307511


In [7]:
pd.read_sql("""
SELECT *
FROM application_train
LIMIT 5;
""", conn)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


# Create a clean, rational, time-aware SQL data model on top of the raw tables.

**Core cleaning rules**

- Convert common “missing” tokens to NULL: '', 'NA', 'NaN', 'null'

- Trim strings: TRIM(col)

- Cast numeric: CAST(col AS REAL/INTEGER)

- Parse dates if they exist (Home Credit often uses day offsets like DAYS_BIRTH, not real dates)

Table List:

- application_train

- application_test

- bureau

- bureau_balance

- credit_card_balance

- installments_payments

- POS_CASH_balance

- previous_application

- (sample_submission doesn’t matter for modeling)

- (HomeCredit_columns_description is metadata)

## application_train

In [11]:
pd.read_sql("PRAGMA table_info(application_train);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,SK_ID_CURR,INTEGER,0,None,0
1,1,TARGET,INTEGER,0,None,0
2,2,NAME_CONTRACT_TYPE,TEXT,0,None,0
3,3,CODE_GENDER,TEXT,0,None,0
4,4,FLAG_OWN_CAR,TEXT,0,None,0
...,...,...,...,...,...,...
117,117,AMT_REQ_CREDIT_BUREAU_DAY,REAL,0,None,0
118,118,AMT_REQ_CREDIT_BUREAU_WEEK,REAL,0,None,0
119,119,AMT_REQ_CREDIT_BUREAU_MON,REAL,0,None,0
120,120,AMT_REQ_CREDIT_BUREAU_QRT,REAL,0,None,0


In [13]:
cursor = conn.cursor()

cursor.execute("""
CREATE VIEW IF NOT EXISTS stg_application_train AS
SELECT
    CAST(NULLIF(SK_ID_CURR, '') AS INTEGER) AS sk_id_curr,

    -- target only exists in train
    CAST(NULLIF(TARGET, '') AS INTEGER) AS target,

    -- numeric examples
    CAST(NULLIF(AMT_INCOME_TOTAL, '') AS REAL) AS amt_income_total,
    CAST(NULLIF(AMT_CREDIT, '') AS REAL) AS amt_credit,
    CAST(NULLIF(AMT_ANNUITY, '') AS REAL) AS amt_annuity,

    -- text cleanup
    NULLIF(TRIM(NAME_CONTRACT_TYPE), '') AS name_contract_type,
    NULLIF(TRIM(CODE_GENDER), '') AS code_gender,

    -- keep all remaining raw columns (optional)
    *
FROM raw_application_train;
""")

conn.commit()